# PKI

The Public Key Infrastructure (PKI) is a system that addresses the problem of verifying the ownership of a public key in public key cryptography. It is a practical solution that ensures the authenticity of a public key and its claimed owner. This lab aims to give students first-hand experience with PKI by providing a series of tasks that focus on the concept. By completing these tasks, students will gain a deeper understanding of how PKI works and how it is used to protect communications. They will also learn how to create and analyze certificates, the potential for man-in-the-middle attacks and how PKI can defeat them. 


In this lab, rather than relying on commercial Certificate Authorities (CA) like VeriSign, we will be creating our own digital certificates. We will become a "root CA" ourselves and use this CA to issue certificates for servers and other entities. This process is known as self-signing. The root CA's certificate is a special type of certificate that is self-signed and is considered unconditionally trusted by most operating systems, web browsers and other software that rely on Public Key Infrastructure (PKI). This task of becoming a root CA and generating a certificate for it is an important step in understanding how digital certificates and PKI work and how we can establish trust in a digital environment without relying on commercial CAs.


In this lab we will be working with the OpenSSL toolkit. This is a widely used open-source implementation of the Secure Sockets Layer (SSL) and Transport Layer Security (TLS) protocols. The manual page for openssl.conf can be found at https://www.openssl.org/docs/man1.0.2/man1/ or other online sources.


#### Configuration

***The first part of this lab must be done using the Linux command shell***
    
By default, OpenSSL uses the configuration file located at "/usr/lib/ssl/openssl.cnf". This file contains settings and options that are used by various OpenSSL commands, including the commands to create and examine digital certificates (ca, req, and x509). In order to allow customization of this file,
we will be working with a copy that we can modify without disrupting any other services or applications running on the system.<br>

In this file there is a section \[CA_default\]

```
[ CA_default ]
dir             = /home/__your_user_name__/demoCA       # Where everything is kept
certs           = $dir/certs     # Where the issued certs are kept
crl_dir         = $dir/crl       # Where the issued crl are kept
database        = $dir/index.txt # database index file.
#unique_subject = no             # Set to ’no’ to allow creation of
                                 # several certs with same subject.
new_certs_dir  = $dir/newcerts   # default place for new certs.
serial         = $dir/serial     # The current serial number
```

<div style="background-color:rgba(255.255.255,.98); color:black; font-weight:normal; padding:6px;"><span style="color:black">
Open up a Linux command shell:<br><br>
* Locate this file and copy it to your working directory for the lab.<br>
* Edit the copy: <br>
&nbsp;&nbsp;&nbsp;&nbsp;- uncomment the "unique_subject" line<br>
&nbsp;&nbsp;&nbsp;&nbsp;- uncomment the "copy_extensions" line <br>
&nbsp;&nbsp;&nbsp;&nbsp;- <i>note: this will allow copying certificate extensions into the final certificate. For security reasons, this is not enabled by default.</i><br>
* Create a directory demoCA<br>
* In demoCA create an empty file named "index.txt"<br>
* In demoCA create a file named "serial" with the value 1000<br>
</div>
    
Once complete, return to the notebook.

<div class="alert alert-block alert-warning">
<span style="color:black">             
NOTE: You must run each cell for the code in it to have effect. You can run them multiple times, or even copy them to new cells if you like.
    
The simplest way to do this is by pressing &lt;SHIFT-ENTER&gt; to run and move to next cell, or &lt;CTRL-ENTER&gt; to run and stay on the current cell.
    </div>  

## Certificates
---
### Creating a Self-Signed CA
Run the openssl command below to request a new certificate. As given, all required information is provided in the command. If you did not want to specify any of this on the command line, you could run it interactively in a command shell.<br><br>
Change the `-subj` argument to specify your chosen country (C), organization (O), and common name (CN). The text you put after "pass:" will be your password. 

This will create two files ca.key and ca.crt. The ca.key file contains the CA's private key. The ca.crt file is the CA's public certificate.


<code>openssl req -x509 -newkey rsa:4096 -sha256 -days 3650 \
    -keyout ca.key -out ca.crt \
    -subj "/CN=www.cybr504CA.com/O=CYBR504 CA/C=US" \
    -passout pass:__ca_password__
</code>   <br>
modify the line below by changing the fields between "___"s

In [ ]:
openssl req -x509 -newkey rsa:4096 -sha256 -days 3650 \
                  -keyout ca.key \
                  -out ./ca.crt \
                  -subj "__subject_line__" \
                  -passout pass:__ca_password__

<br>Notice
 * How many bits is the key?<br>
 * When will the certificate expire?

Run the openssl command
<code>openssl x509 -in ca.crt -text -noout</code>
to examine the public certificate.

<i>The -text option displays the output as text, the -noout option prevents displaying the encoded version.</i>

<br>How can you tell that this is a CA certificate?

How can you tell that this ia a self-signed certificate?

In a new cell, use a modified version of the "openssl x509" command you used before to see the RSA key.

<i>Hint: Change the x509 to rsa to see the rsa key. You'll also need to specify the correct password. Use the -passin pass:????? option for that.</i><br>

<i>Note: if you don't, the cell will get stuck waiting for input. You can tell if this is the problem by noticing the black circle at the top right of the lab window. Use the square stop icon to break out of it.</i>

<br>RSA keys are generated using an a public exponent, a private exponent, a modulus n and two secret numbers p and q.
- Are these stated in the certificate? 
- Are they stated in the key?

Now let's use our CA to sign another key. That will require a CSR, a Certificate Signing Request. Like the CA certificate, the CSR will have identity informaiton about the requestor and the company's public key. Once the CSR has been generated, it must be sent to a CA to verify it, and if satisfied, to sign it.

We'll use a similar openssl command as before, except by removing the -x509 option it will create a CSR. Note also, there is no expiration date option. This is provided by the signer.<br>
<code>openssl req -newkey rsa:2048 -sha256 -keyout server.key -out server.csr -subj  "__subject_line__" -passout pass:__csr-pw__</code>

Modify as needed.

Review the CSR and key. Do this for each.<br>
&nbsp;&nbsp;&nbsp;&nbsp;for the CSR: <code>openssl req -in <i>\<file\></i> -text -nout</code><br>
&nbsp;&nbsp;&nbsp;&nbsp;for the key: <code>openssl req -in <i>\<file\></i> -text -nout -passin pass:<i>\<password\></i></code><br>

In [70]:
#CSR

In [71]:
#key

<br>
How can you tell if a certificate is a CSR?

### Including Alternative Names

Websites often have multiple URLs that redirect to the same website. For example, www.cybr504.com, cybr504.com, cybr504.net, and usdcrypto.info could all lead to the same website. However, due to browser hostname matching policies, the common name on a certificate must match the server's hostname, or the browser will not establish a connection. To allow for multiple names on a single certificate, the X.509 specification includes an extension called the "Subject Alternative Name" (SAN). Using this extension, it's possible to list multiple hostnames in the "subjectAltName" field of a certificate.<br><Br>
To create a certificate signing request with this field, you can add all necessary information to a configuration file or through the command line. It's important to note that the subjectAltName field must also include the common name, or the common name will not be recognized as a valid name.

```
-addext "subjectAltName = DNS:www.cybr504.com, \
                          DNS:www.cybr504.net, \
                          DNS:www.cybr504m5.com"
```

Use the command you used before to create the CSR, but make if to include alterative names.

### Signing the CSR
The CSR must be signed by a CA to become a certificate. In practice, an organization would send their CSR to their chosen CA for signing. In this lab we will sign it directly.

```
openssl ca -config __openssl_config_file__ -policy policy_any \
-md sha256 -days 3650 \
-in server.csr -out server.crt -batch \
-cert ca.crt -keyfile ca.key \
-passin pass:__ca_password__
```
The policy we are utilizing, known as "policy anything," is specified in our configuration file. This policy is not the standard option, as the default policy enforces stricter guidelines, mandating that certain elements of the subject information in the request align with those in the certificate of the CA. However, the policy specified in our command, as indicated by its name, does not impose any matching requirements.

In [ ]:
openssl ca -config __your_openssl.cnf_file__ -policy policy_anything \
-md sha256 -days 3650 \
-in server.csr -out server.crt -batch \
-cert ca.crt -keyfile ca.key \
-passin pass:__ca_password__

In [ ]:
openssl x509 -in server.crt -text -noout

Check the server certificate. 
* Is it a CA? Why?
* What is listed in the "Subject Alternative Name" section?
* Who is listed as the Issuer?

---
### This concludes the lab.

Use File > Save and Export Notebook as... > HTML